In [1]:
import pandas as pd 
import numpy as np
import calendar

from datetime import datetime, date, timedelta
from math import sqrt


In [2]:
import ccxt
exchange_class = getattr(ccxt, 'binance')
binance  = exchange_class()


def min_ohlcv(dt, pair, limit):
    # UTC native object
    since = calendar.timegm(dt.utctimetuple())*1000
    ohlcv1 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv2 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv = ohlcv1 + ohlcv2
    return ohlcv

def ohlcv(dt, pair, period='1d'):
    ohlcv = []
    limit = 1000
    if period == '1m':
        limit = 720
    elif period == '1d':
        limit = 365
    elif period == '1h':
        limit = 24
    elif period == '5m':
        limit = 288
    for i in dt:
        start_dt = datetime.strptime(i, "%Y%m%d")
        since = calendar.timegm(start_dt.utctimetuple())*1000
        if period == '1m':
            ohlcv.extend(min_ohlcv(start_dt, pair, limit))
        else:
            ohlcv.extend(binance.fetch_ohlcv(symbol=pair, timeframe=period, since=since, limit=limit))
    df = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
    df['Open'] = df['Open'].astype(np.float64)
    df['High'] = df['High'].astype(np.float64)
    df['Low'] = df['Low'].astype(np.float64)
    df['Close'] = df['Close'].astype(np.float64)
    df['Volume'] = df['Volume'].astype(np.float64)
    df.set_index('Time', inplace=True)
    return df

from datetime import datetime, date, timedelta

start_day = "20220101" #"20201201"
start_dt = datetime.strptime(start_day, "%Y%m%d")
end_day = "20220630"
end_dt = datetime.strptime(end_day, "%Y%m%d")
days_num = (end_dt - start_dt).days + 1
datelist = [start_dt + timedelta(days=x) for x in range(days_num)]
datelist = [date.strftime("%Y%m%d") for date in datelist]

#fwd = 10
coin = 'ETH'
pair = coin + "/USDT"
df = ohlcv(datelist, pair, '1m')

df.to_csv("prices_all_1m.csv")

In [3]:
def calculateL(pNow, pDeposit, vDeposit ,pLower, pUpper) : 
    if pDeposit < pLower : 
        L = vDeposit / (pDeposit * ( 1 / sqrt(pLower) - 1 / sqrt(pUpper)))
    elif pDeposit > pUpper :
        L = vDeposit / (sqrt(pUpper) - sqrt(pLower) )
    else : 
        L = vDeposit / (pDeposit*(1 / sqrt(pDeposit) - 1 / sqrt(pUpper) ) + sqrt(pDeposit) - sqrt(pLower) )

    return L 

def getYLower(pLower, pUpper, L) :
    return L * sqrt(pLower)

def getXUpper(pLower, pUpper, L) :
    return L / sqrt(pUpper) 
    
def getYMax(pLower, pUpper, L) :
    return (sqrt(pUpper) - sqrt(pLower)) * L 
    
def getXMax(pLower, pUpper, L) : 
    return (1 / sqrt(pLower) - 1 / sqrt(pUpper)) * L 
    
def getXReal(pCurrent, pLower, pUpper, L) : 
    if pCurrent < pLower : 
        X = getXMax(pLower, pUpper, L)
    elif pCurrent > pUpper :
        X = 0
    else : 
        X = (L / sqrt(pCurrent)) - getXUpper(pLower, pUpper, L)
    return X 
    
def getYReal(pCurrent, pLower, pUpper, L) : 
    if pCurrent < pLower : 
        Y = 0
    elif pCurrent > pUpper :
        Y = getYMax(pLower, pUpper, L)
    else : 
        Y = (L * sqrt(pCurrent)) - getYLower(pLower, pUpper, L)

    return Y     
    
def getXLP(pCurrent, pLower, pUpper, L) : 
    return (getXReal(pCurrent, pLower, pUpper, L) * pCurrent)

def getYLP(pCurrent, pLower, pUpper, L) : 
    return (getYReal(pCurrent, pLower, pUpper, L))
    
def getVLP(pCurrent, pLower, pUpper, L) : 
    x = getXLP(pCurrent, pLower, pUpper, L)
    y = getYLP(pCurrent, pLower, pUpper, L)
    return (x + y)



In [4]:

lendAllocation = .65
#debt thresholds for rebalancing
debtLow = .975
debtHigh = 1.025
collatLow = .5
collatTarget = .55
collatHigh = .6
slippageAdj = 0 #.0005 #when rebalancing losses from swapping 

# APY's
lendRate = .01
borrowRate = .03
lpRewards = 1.5

#SIM INFO
stepsPerYear = 365*24*60 #if simulating hourly set to be 365 * 24 

annualRates = {'lend' : lendRate, 'borrow' : borrowRate, 'lp' : lpRewards}
adjRates = {'lend' : ((1 + lendRate)**(1/stepsPerYear) - 1), 
            'borrow' : ((1 + borrowRate)**(1/stepsPerYear) - 1),
            'lp' : ((1 + lpRewards)**(1/stepsPerYear) - 1)}

# lpRange 
lowRange = 0.25
highRange = 4


In [5]:
def doRebalance(cRatio, debtRatio) : 
    if cRatio < collatLow : doRebalance = True
    elif cRatio > collatHigh : doRebalance = True
    elif debtRatio < debtLow : doRebalance = True
    elif debtRatio > debtHigh : doRebalance = True
    else : doRebalance = False
    
    return doRebalance


def enterPosition(pNow, depositAmount):
    Collateral = lendAllocation*depositAmount
    Debt = (1 - lendAllocation)*depositAmount
    lpAmount = Debt*2
    DebtX = Debt / pNow
    lpAmount = Debt*2 # we pair equally     
    PLow = lowRange * pNow
    PHigh = highRange * pNow
    PEntry = pNow 
    L = calculateL(pNow, pNow, lpAmount ,PLow, PHigh)
    return (PEntry, Collateral, DebtX, PLow, PHigh, L, lpAmount)

def updatePosition(p, Collateral, DebtX):

    Collateral = Collateral*(1+adjRates['lend'])
    DebtX = DebtX *(1+adjRates['borrow'])
    Debt = DebtX * p
    return Collateral, DebtX, Debt

#def exitPosition

In [6]:
len(df)

260640

In [7]:
PStart = df['Open'].iloc[0]
depositAmount = 1000
PEntry, Collateral, DebtX, PLow, PHigh, L, lpAmount = enterPosition(PStart, depositAmount)

maxPortfolio = depositAmount 
minPortfolio = depositAmount

harvestFrequency = 360 

portfolioTrack = []
trackFreq = 60*24

for i in range(len(df)) : 
        p = df['Close'].iloc[i]
        Collateral, DebtX, Debt = updatePosition(p, Collateral, DebtX)
        
        #Debt = DebtX * p
        L = calculateL(p, PEntry, lpAmount ,PLow, PHigh)           
        # add rewards # 
        if i % harvestFrequency == 0 : 
            #print("Harvest ---- ")
            L = L *(1+adjRates['lp']*harvestFrequency)
        
        # Key Metrics for rebalancing 
        
        CRatio = Debt / Collateral 
        xInLp = getXReal(p, PLow, PHigh, L)
        DebtRatio = xInLp / DebtX
        
        # rebalance LOGIC HERHEHEHREHERHEEH
        rebalance = doRebalance(CRatio, DebtRatio)
        if rebalance :
            #print("Rebalance ---- ")
            #print("C Ratio : "  + str(int(CRatio*100)))
            #print("Debt Ratio : "  + str(int(DebtRatio*100)))

            lpValue = getVLP(p, PLow, PHigh, L)
            if DebtRatio < 1 : 
                slippage = (DebtX - xInLp)*p*slippageAdj
            else : 
                slippage = (xInLp - DebtX)*p*slippageAdj
            portFolio = Collateral - Debt + lpValue - slippage 
            
            # we re-enter position with new range 
            PEntry, Collateral, DebtX, PLow, PHigh, L, lpAmount = enterPosition(p, portFolio)
            
        lpValue = getVLP(p, PLow, PHigh, L)
        # PRINT SOME FREE MONEY REEEEEEE
        portFolio = Collateral - Debt + lpValue

        maxPortfolio = max(maxPortfolio, portFolio)
        minPortfolio = min(minPortfolio, portFolio)
    
        if i % trackFreq == 0 : 
            print("Iteration Number --- " + str(i))
            print("Price "  + str(int(p)))
            print("Portfolio Value --- " + str(int(portFolio)))
            portfolioTrack.append(portFolio)
            print(" --- " )


Iteration Number --- 0
Price 3684
Portfolio Value --- 1000
 --- 
Iteration Number --- 1440
Price 3766
Portfolio Value --- 1000
 --- 
Iteration Number --- 2880
Price 3823
Portfolio Value --- 1000
 --- 
Iteration Number --- 4320
Price 3773
Portfolio Value --- 1000
 --- 
Iteration Number --- 5760
Price 3788
Portfolio Value --- 1000
 --- 
Iteration Number --- 7200
Price 3535
Portfolio Value --- 1021
 --- 
Iteration Number --- 8640
Price 3415
Portfolio Value --- 999
 --- 
Iteration Number --- 10080
Price 3206
Portfolio Value --- 998
 --- 
Iteration Number --- 11520
Price 3077
Portfolio Value --- 1013
 --- 
Iteration Number --- 12960
Price 3147
Portfolio Value --- 998
 --- 
Iteration Number --- 14400
Price 3086
Portfolio Value --- 998
 --- 
Iteration Number --- 15840
Price 3243
Portfolio Value --- 978
 --- 
Iteration Number --- 17280
Price 3369
Portfolio Value --- 984
 --- 
Iteration Number --- 18720
Price 3242
Portfolio Value --- 1011
 --- 
Iteration Number --- 20160
Price 3315
Portfolio Va

Iteration Number --- 188640
Price 2089
Portfolio Value --- 1013
 --- 
Iteration Number --- 190080
Price 1967
Portfolio Value --- 964
 --- 
Iteration Number --- 191520
Price 2015
Portfolio Value --- 972
 --- 
Iteration Number --- 192960
Price 2055
Portfolio Value --- 963
 --- 
Iteration Number --- 194400
Price 2140
Portfolio Value --- 951
 --- 
Iteration Number --- 195840
Price 2024
Portfolio Value --- 962
 --- 
Iteration Number --- 197280
Price 2088
Portfolio Value --- 962
 --- 
Iteration Number --- 198720
Price 1921
Portfolio Value --- 981
 --- 
Iteration Number --- 200160
Price 2020
Portfolio Value --- 945
 --- 
Iteration Number --- 201600
Price 1957
Portfolio Value --- 973
 --- 
Iteration Number --- 203040
Price 1976
Portfolio Value --- 962
 --- 
Iteration Number --- 204480
Price 2039
Portfolio Value --- 962
 --- 
Iteration Number --- 205920
Price 1975
Portfolio Value --- 977
 --- 
Iteration Number --- 207360
Price 1977
Portfolio Value --- 962
 --- 
Iteration Number --- 208800
Price